In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL du site
base_url = "https://www.digischool.fr"

# URLs par établissement
urls_depart = [
    "https://www.digischool.fr/primaire",
    "https://www.digischool.fr/college",
    "https://www.digischool.fr/lycee"
]

def get_links(url : str, prefix : str) -> list:
    """
    Fonction pour récupérer les liens des cours.

    Args:
        url (str): URL de la page.
        prefix (str): Préfixe des liens des cours.
    
    Returns:
        list: Liste des liens des cours.
    """

    # Récupération de la réponse de la requête
    response = requests.get(url)
    liens = []

    # Si la réponse est 200, on parse le contenu de la page pour récupérer les liens des cours pour un niveau donné
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        balises_a = soup.find_all('a', href=True)
        liens_filtres = [a['href'] for a in balises_a if a['href'].startswith(prefix)]
        liens_complets = [base_url + lien for lien in liens_filtres]
        liens.extend(liens_complets)
    return liens

def get_lesson(url : str, cours_data : str):
    """
    Fonction pour récupérer le contenu des cours.

    Args:
        url (str): URL de la page.
        cours_data (list): Liste pour stocker les données des cours.
    """

    # Récupération de la réponse de la requête
    response = requests.get(url)

    # Si la réponse est 200, on parse le contenu de la page pour récupérer les liens des cours d'une matière donnée
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        balises_a = soup.find_all('a', href=True)
        liens_cours = [a['href'] for a in balises_a if a['href'].startswith('/cours/')]
        liens_complets = ["https://www.digischool.fr" + lien for lien in liens_cours]
        
        for lien in liens_complets:
            # Extraction du nom du cours
            partie_cours = lien.split('/cours/')[1]
            nom_cours = partie_cours.split('?')[0].replace('-', ' ')
            
            # Extraction du nom du chapitre
            if '%2C' in lien:
                nom_chapitre = lien.split('%2C')[-1].replace('-', ' ')
            else:
                nom_chapitre = "N/A"
            
            # Extraction du nom de la matière et du niveau
            parties = lien.split('%2C')
            if len(parties) >= 3:
                nom_matiere = parties[-2].replace('-', ' ')
                niveau = parties[-3].replace('-', ' ')
            else:
                nom_matiere = "N/A"
                niveau = "N/A"
            
            # Récupération de la réponse de la requête
            second_response = requests.get(lien)
            
            # Si la réponse est 200, on parse le contenu de la page pour récupérer le contenu des cours
            if second_response.status_code == 200:
                second_soup = BeautifulSoup(second_response.text, 'html.parser')
                span_latex = second_soup.find('span', class_='__Latex__')
                if span_latex:
                    article = span_latex.find_parent('article')
                    if article:
                        texte_nettoye = article.get_text(separator=' ', strip=True)
                    else:
                        texte_nettoye = "Balise </article> non trouvée."
                else:
                    texte_nettoye = "Balise <span class=\"__Latex__\"> non trouvée."
            else:
                texte_nettoye = "Erreur lors de la récupération de la page."
            
            # Stockage des informations
            cours_info = {
                "lien": lien,
                "nom_du_cours": nom_cours,
                "nom_du_chapitre": nom_chapitre,
                "nom_de_la_matiere": nom_matiere,
                "niveau": niveau,
                "texte": texte_nettoye
            }
            
            cours_data.append(cours_info)
        
        # Mise en forme des informations
        if len(liens_cours) > 0:
            exemple_lien = liens_cours[0]
            parties_exemple = exemple_lien.split('%2C')
            if len(parties_exemple) >= 3:
                nom_matiere_ex = parties_exemple[-2].replace('-', ' ')
                niveau_ex = parties_exemple[-3].replace('-', ' ')
            else:
                nom_matiere_ex = "N/A"
                niveau_ex = "N/A"
            print(f"[INFO] Tous les cours de '{nom_matiere_ex}' de niveau '{niveau_ex}' ont été récupérés.")
    else:
        print("Erreur lors de la récupération de la page.")

def main():
    """
    Fonction principale.
    """

    # Définition de la structure des liens
    toutes_les_pages = []
    for url in urls_depart:
        if '/primaire' in url:
            prefix = '/primaire/'
        elif '/college' in url:
            prefix = '/college/'
        elif '/lycee' in url:
            prefix = '/lycee/'
        else:
            prefix = '/'
        liens = get_links(url, prefix)
        toutes_les_pages.extend(liens)
    
    print(toutes_les_pages)
    # cours_data = []

    # # Itération sur les pages pour récupérer les cours
    # for page in toutes_les_pages:
    #     get_lesson(page, cours_data)
    
    # # Conversion des données en DataFrame et exportation en CSV
    # df = pd.DataFrame(cours_data)
    # df.to_csv('cours.csv', index=False, sep="|", encoding="utf-8")
    # print("[FINISH] Les données ont été exportées dans 'cours.csv'.")

if __name__ == "__main__":
        main()

['https://www.digischool.fr/primaire/activation', 'https://www.digischool.fr/primaire/cp', 'https://www.digischool.fr/primaire/cp/francais', 'https://www.digischool.fr/primaire/cp/mathematiques', 'https://www.digischool.fr/primaire/cp/questionner-le-monde', 'https://www.digischool.fr/primaire/cp/emc', 'https://www.digischool.fr/primaire/ce1', 'https://www.digischool.fr/primaire/ce1/francais', 'https://www.digischool.fr/primaire/ce1/mathematiques', 'https://www.digischool.fr/primaire/ce1/questionner-le-monde', 'https://www.digischool.fr/primaire/ce1/emc', 'https://www.digischool.fr/primaire/ce2', 'https://www.digischool.fr/primaire/ce2/francais', 'https://www.digischool.fr/primaire/ce2/mathematiques', 'https://www.digischool.fr/primaire/ce2/questionner-le-monde', 'https://www.digischool.fr/primaire/ce2/emc', 'https://www.digischool.fr/primaire/cm1', 'https://www.digischool.fr/primaire/cm1/francais', 'https://www.digischool.fr/primaire/cm1/mathematiques', 'https://www.digischool.fr/prima

In [40]:
def afficher_html(url: str):
    """
    Fonction pour afficher le code HTML d'une page à partir du lien.

    Args:
        url (str): URL de la page.
    """
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        print(soup.prettify())
    else:
        print(f"Erreur lors de la récupération de la page. Code de statut: {response.status_code}")

# Exemple d'utilisation
afficher_html("https://www.digischool.fr/lycee")

<!DOCTYPE html>
<html class="tw-scroll-smooth" data-theme="lycee" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <script type="application/ld+json">
   {"@context":"https://schema.org","@type":"SoftwareApplication","name":"digiSchool","operatingSystem":"Web, IOS, Android","applicationCategory":"EducationalApplication","offers":{"@type":"Offer","price":"0","priceCurrency":"EUR"},"aggregateRating":{"@type":"AggregateRating","reviewCount":"8789","ratingValue":"4.6"}}
  </script>
  <meta content="summary_large_image" name="twitter:card"/>
  <meta content="@site" name="twitter:site"/>
  <meta content="@handle" name="twitter:creator"/>
  <meta content="421248398000974" property="fb:app_id"/>
  <meta content="https://www.digischool.fr/lycee" property="og:url"/>
  <meta content="website" property="og:type"/>
  <meta content="fr_FR" property="og:locale"/>
  <meta content="digiSchool" property="og:site_name"/>
  <link href="https://www.digis